In [2]:
import numpy as np
import torch
import os
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Load the data from the specified directory
X_train = np.load('ProcessedInputData/X_train.npy')
Y_train = np.load('ProcessedInputData/y_train.npy')
X_val = np.load('ProcessedInputData/X_val.npy')
Y_val = np.load('ProcessedInputData/y_val.npy')

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
Y_val_tensor = torch.tensor(Y_val, dtype=torch.long)

# Create a DataLoader for batching
batch_size = 64
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(X_val_tensor, Y_val_tensor)
val_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

In [15]:
# Print the first few elements of the tensors
print("First few elements of Y_train_tensor:")
print(Y_train_tensor.size())
print(Y_train_tensor[:10])  # Print first 10 elements

print("First few elements of X_train_tensor:")
print(X_train_tensor.size())
# print(X_train_tensor[:3])  # Print first 3 elements

# each input is 5x90x1 matrix
# 5 bc there are 5 eeg channels recordings per sample,
# 90 bc i did fourier transform and binned it into 90 different bins from 0.5 to 40 Hz (relevant frequencies for EEG)
# the 1 was added just to be the dimension of picture style for CNN so its like black and white

print("Val tensors:")
print(Y_val_tensor.size())
print(X_val_tensor.size())

First few elements of Y_train_tensor:
torch.Size([3153])
tensor([1, 1, 1, 1, 0, 0, 0, 1, 0, 0])
First few elements of X_train_tensor:
torch.Size([3153, 5, 90, 1])
Val tensors:
torch.Size([394])
torch.Size([394, 5, 90, 1])


In [16]:
sanity_dataset = TensorDataset(X_train_tensor[:20], Y_train_tensor[:20]) # take just a few elements
sanity_loader = DataLoader(sanity_dataset, batch_size=5, shuffle=True)

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform_(m.weight)  # Xavier initialization
        if m.bias is not None:
            nn.init.zeros_(m.bias)  # Initialize biases to zero

In [36]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(3, 3), padding=1)  
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(3, 3), padding=1)  
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=2)  
        self.relu = nn.ReLU()

        # Compute correct FC input size dynamically
        self._to_linear = None
        self._compute_linear_input_size()

        self.fc1 = nn.Linear(self._to_linear, 64)  
        self.fc2 = nn.Linear(64, 1)  
        

    def _compute_linear_input_size(self):
        with torch.no_grad():
            x = torch.zeros(1, 1, 5, 90)  # Dummy input
            x = self.pool(self.relu(self.conv1(x)))  
            x = self.pool(self.relu(self.conv2(x)))  
            self._to_linear = x.numel()  # Compute feature size

    def forward(self, x):
        x = x.view(-1, 1, 5, 90)  
        x = self.pool(self.relu(self.conv1(x)))  
        x = self.pool(self.relu(self.conv2(x)))  
        x = x.view(x.size(0), -1)  # Flatten dynamically
        x = self.relu(self.fc1(x))  
        x = self.fc2(x)  
        return x  # No sigmoid!

# Training Function
def train_model(model, train_loader, epochs=10):
    torch.manual_seed(1000)
    model.train()
    criterion = nn.BCEWithLogitsLoss()  
    optimizer = optim.SGD(model.parameters(), lr=0.001)  # Use Adam

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            labels = labels.view(-1, 1).float()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Convert logits to probabilities and round
            predicted = torch.round(torch.sigmoid(outputs))  
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        avg_loss = running_loss / len(train_loader)
        accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')

# Initialize Model
cnn_model = CNN()
cnn_model.apply(init_weights)  # Apply Xavier Initialization

# Train Model
train_model(model=cnn_model, train_loader=sanity_loader, epochs=50)

Epoch [1/50], Loss: 115.4306, Accuracy: 55.00%
Epoch [2/50], Loss: 3.9472, Accuracy: 50.00%
Epoch [3/50], Loss: 29.1186, Accuracy: 65.00%
Epoch [4/50], Loss: 0.9319, Accuracy: 60.00%
Epoch [5/50], Loss: 0.6777, Accuracy: 60.00%
Epoch [6/50], Loss: 0.5764, Accuracy: 65.00%
Epoch [7/50], Loss: 0.5146, Accuracy: 65.00%
Epoch [8/50], Loss: 0.4705, Accuracy: 65.00%
Epoch [9/50], Loss: 0.4359, Accuracy: 65.00%
Epoch [10/50], Loss: 0.4068, Accuracy: 75.00%
Epoch [11/50], Loss: 0.3833, Accuracy: 80.00%
Epoch [12/50], Loss: 0.3665, Accuracy: 95.00%
Epoch [13/50], Loss: 0.3558, Accuracy: 95.00%
Epoch [14/50], Loss: 0.3475, Accuracy: 95.00%
Epoch [15/50], Loss: 0.3362, Accuracy: 95.00%
Epoch [16/50], Loss: 0.3292, Accuracy: 95.00%
Epoch [17/50], Loss: 0.3219, Accuracy: 95.00%
Epoch [18/50], Loss: 0.3188, Accuracy: 90.00%
Epoch [19/50], Loss: 0.3163, Accuracy: 95.00%
Epoch [20/50], Loss: 0.2949, Accuracy: 100.00%
Epoch [21/50], Loss: 0.3075, Accuracy: 95.00%
Epoch [22/50], Loss: 0.3345, Accuracy: 